In [60]:
import pandas as pd
import numpy as np

data = pd.read_excel("../EDA/Datasets/ad-data-cleaned.xlsx")

# Impute missing values in the 'days_since_last_punishment' column with 91
data.fillna({'days_since_last_punishment': 91}, inplace=True)

# Ad Scoring System
Before implementing the Ad Scoring System, two new features will be created

1. `days_diff`: Number of days between the ad being uploaded on Ads Manager and when the advertiser wants the ad to start
- Derived from `start_time` - `pdate`
- If negative (which occurs for almost half of the data), impute with mean number of days among the positive values
- Should have an inverse relationship with the ad score, the lower the `days_diff` the more urgent the ad is, hence the higher priority

2. `tier`: Advertiser Tier
- The `tier_score` of each advertiser is derived from the formula below:

$$
\text{tier score} = \beta_1 \times \text{avg ad revenue} + \beta_2 \times \text{punish num} + \beta_3 \times \text{days since last punishment}
$$

The coefficients $\beta_1$, $\beta_2$ and $\beta_3$ represent weights that determine the importance of the features above. These coefficients ideally should be determined from calibrating and optimizing based on feedback and desired outcomes. However, for this we will assume $\beta_1$ = 0.5, and $\beta_2$ = $\beta_3$ = 0.25

- A unique score is derived for every advertiser (which can be found from unique `avg_ad_revenue`) and tier thresholds are calculated based on quantiles
- Each advertiser is then assigned a `tier` from 1-10

In [66]:
# Calculate days_diff
data['days_diff'] = (data['start_time'] - data['p_date']).dt.days

# Replace negative values with the mean of positive values
mean_positive_days_diff = data[data['days_diff'] > 0]['days_diff'].mean()
data['days_diff'] = data['days_diff'].apply(lambda x: mean_positive_days_diff if x < 0 else x)


In [67]:
# Impute NaN values in the 'days_diff' column
data['days_diff'].fillna(mean_positive_days_diff, inplace=True)

In [63]:
# Calculate the new score for each advertiser using the revised formula
data['tier_score'] = 0.5 * data['avg_ad_revenue'] - 0.25 * data['punish_num'] + 0.25 * data['days_since_last_punishment']

# Group by advertiser (based on avg_ad_revenue) and get the unique score for each advertiser
unique_advertisers_scores = data.drop_duplicates(subset='avg_ad_revenue')[['avg_ad_revenue', 'tier_score']]

# Define tier thresholds based on quantiles for the new scores
tier_thresholds = [unique_advertisers_scores['tier_score'].quantile(i/10) for i in range(1, 11)]

# Assign tiers to each advertiser based on their score
def assign_tier(score):
    for i, threshold in enumerate(tier_thresholds):
        if score <= threshold:
            return i+1
    return 10

# Assign tiers to each advertiser based on their new score
unique_advertisers_scores['tier'] = unique_advertisers_scores['tier_score'].apply(assign_tier)

# Merge the tiers from the unique_advertisers_scores dataframe to the original data dataframe
data = data.merge(unique_advertisers_scores[['avg_ad_revenue', 'tier']], on='avg_ad_revenue', how='left')

# Gurobipy Optimization Model (Ads)
1. **Decision Variable**
- For each ad, $x_i$ represents the priority score for the i-th ad

2. **Objective Function**
- As we define priority score to be a product of `avg_ad_revenue`, `baseline_st`, `days_diff` (negative correlation) and `tier`, the optimization model will aim to maximize priority score
- Hence, the objective function is defined below
$$
\sum_{i=1}^{n} \left( \beta_1 \times \text{avg\_ad\_revenue}_i + \beta_2 \times \text{baseline\_st}_i - \beta_3 \times \text{days\_diff}_i + \beta_4 \times \text{tier}_i \right) x_i
$$

- However, we need to combine the above objective function with a quadratic term to model a normal distribution around from 0-1. The quadratic objective component is below
$$
\sum_{i=1}^{n} \left( x_i - 0.5 \right)^2
$$

- Combining the two objective functions, the final objective function is obtained below
$$
\sum_{i=1}^{n} \left( \beta_1 \cdot \text{avg\_ad\_revenue}_i + \beta_2 \cdot \text{baseline\_st}_i - \beta_3 \cdot \text{days\_diff}_i + \beta_4 \cdot \text{tier}_i \right) x_i - \lambda \sum_{i=1}^{n} \left( x_i - 0.5 \right)^2
$$
where the coefficients $\beta$ and $\lambda$ are derived from an iterative training and optimization process of the Gurobi model. By analyzing the results over multiple iterations and adjusting these coefficients, one can refine the model's performance and achieve better task allocations that meet specific business objectives. However for this, we will assume $\beta_1$ = $\beta_2$ = $\beta_3$ = $\beta_4$ = 0.25 and $\lambda$ = 20

3. Constraints
- Mean of $x_i$ = 0.5

In [64]:
import gurobipy as gp
from gurobipy import GRB

# Create a new Gurobi model
m = gp.Model("ad_priority_optimization")

# Constants
beta = 0.25
n = len(data)

# Add decision variables
x = m.addVars(n, lb=0, ub=1, name="x")

# Weighting factor
lambda_factor = 20

# Original objective
original_obj = gp.quicksum(
    (beta * data['avg_ad_revenue'][i] + beta * data['baseline_st'][i] - beta * data['days_diff'][i] + beta * data['tier'][i]) * x[i]
    for i in range(n)
)

# Quadratic objective component
quadratic_obj = gp.quicksum((x[i] - 0.5)*(x[i] - 0.5) for i in range(n))

# Combined objective
m.setObjective(original_obj - lambda_factor * quadratic_obj, GRB.MAXIMIZE)

# Add constraint: mean(x) = 0.5
m.addConstr(x.sum() / n == 0.5, "mean_constraint")

# Solve the model
m.optimize()

# Check if the model has a solution
if m.status == GRB.Status.OPTIMAL:
    solution_values = [x[i].x for i in range(n)]
else:
    solution_values = []

data['ad_score'] = solution_values

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 39564 columns and 39564 nonzeros
Model fingerprint: 0x998dcdfc
Model has 39564 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-05, 3e-05]
  Objective range  [1e+01, 4e+03]
  QObjective range [4e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 5e-01]
Presolve time: 0.03s
Presolved: 1 rows, 39564 columns, 39564 nonzeros
Presolved model has 39564 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00 (roughly 16 MB of memory)
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.23285838e+10  1.27791719e+10  3.28e+07 6.25e+02  1.00e+06

[0.9999999999999813,
 0.9999999999999104,
 0.4436669975576987,
 0.4420419975577061,
 0.4420419975577061]

**Confidence Metric**

`confidence` would be a new feature added to each advertisement and it represents our model's certainty in its prediction regarding the presence or absence of a violation in an ad. A value close to 1 indicates high certainty in the prediction, whether the ad is identified as having a violation or not, while a value close to 0 indicates low certainty

It would consist of two components:
1. The model confidence determined from our video ___ model. The formula for determining the confidence from the score array of possible violations can be found below.

In [65]:
# Apply min-max normalization to the punish_num column to create the confidence column
min_punish = data['punish_num'].min()
max_punish = data['punish_num'].max()

data['confidence'] = (data['punish_num'] - min_punish) / (max_punish - min_punish)